In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('final_metadata.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df.info()

In [ ]:
df['soup'] = df.apply(lambda row: f"Title: {row['title']}. Genres: {row['genres']}. Keywords: {row['keywords']}. Cast: {row['cast']}. Director: {row['director']}."
                                  f"Released in: {row['release_year']}. ", axis=1)

In [ ]:
from langchain.docstore.document import Document

In [ ]:
movies = []

for index, row in df.iterrows():
    x = Document(page_content=row['soup'],
                 metadata = {
                     "movie": row['title'],
                     "language": row['original_language'],
                     "popularity": row['popularity'],
                     "year": row['year'],
                     "synopsis": row['overview'],
                     "score": row['score'],
                     "poster_path": row['poster_path']
                 })
    movies.append(x)

In [ ]:
import weaviate
WEAVIATE_API_KEY = "AtG7J8fQCMyOEaRJqX3qZpzKTzPWjvQhFqCg"
client = weaviate.Client(
    url="https://qlupgd5jry6mdfnor0d0vw.c0.asia-southeast1.gcp.weaviate.cloud",
    auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY)
)

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
  model_name=embedding_model_name,
  model_kwargs=model_kwargs
)

In [ ]:
import weaviate
from langchain.vectorstores import Weaviate
from langchain_weaviate.vectorstores import WeaviateVectorStore

In [ ]:
vector_db = Weaviate.from_documents(
    movies,
    embeddings,
    client=client,
    by_text=False
)

In [ ]:
soups = pd.Series(df['soup'].values, index=df['title'])